# Problem 3
The infinite matrix $ A $ has entries $ a_{11} = 1, a_{12} = \frac{1}{2}, a_{21} = \frac{1}{3}, a_{13} = \frac{1}{4}, a_{22} = \frac{1}{5}, a_{31} = \frac{1}{6} $ ... is a bounded operator on $ \ell_2$. What is ||A||?

### Finding a general formula for the matrix's terms
We have that

$ A = \begin{bmatrix} 
1   & 1/2 & 1/4 & 1/7 & \cdots \\
1/3 & 1/5 & 1/8 & \cdots & \vdots \\
1/6 & 1/9 & \cdots & \ddots & \vdots \\
1/10 & \cdots & \cdots & \ddots & \vdots \\
\vdots & \vdots & \vdots & \vdots &\vdots & \\
\end{bmatrix} $

Consider the matrix with all of the reciprocals of the elements in A:


$ B = \begin{bmatrix}
1 & 2 & 4 & 7 & 11 & 16 & \cdots \\
3 & 5 & 8 & 12 & 17 & \cdots \\
6 & 9 & 13 & 18 & \cdots \\
10 & 14 & 19 & \cdots \\
15 & 20 \cdots \\
21 \cdots \\
\end {bmatrix} $

Noting that the first column of B is just the triangular numbers, we guess that the general term $B_{ij}$ can be represented as a quadratic in both i and j, i.e. that $ B_{ij} = a i^2 + b ij + c j^2 + d i + e j + f,$ with $i,j\geq0$.

Picking 6 elements from B and solving the resulting system of 6 equations in 6 unknowns, we have that
$ B_{ij} = \frac{1}{2} i^2 + ij + \frac{1}{2}j^2 + \frac{3}{2} i + \frac{1}{2} j + 1, A_{ij} = \frac{1}{B_{ij}}$

### Brute force attempt
We first find the spectral norm of the nxn upper left corner of A for n=1,2,4,...,4096 and see if we can get converging answers to get some accurate digits

In [1]:
from numpy.linalg import norm
A = lambda i,j: 1/(0.5*i*i + i*j + 0.5*j*j + 1.5*i + 0.5*j + 1)
for n in [2**i for i in range(13)]:
    %time print(n, norm([[A(i,j) for i in range(n)] for j in range(n)],2))

1 1.0
CPU times: user 1.49 ms, sys: 918 µs, total: 2.41 ms
Wall time: 22.3 ms
2 1.1833501765516565
CPU times: user 1.04 ms, sys: 0 ns, total: 1.04 ms
Wall time: 1.18 ms
4 1.2525373975167995
CPU times: user 1.12 ms, sys: 0 ns, total: 1.12 ms
Wall time: 1.05 ms
8 1.2700463058540765
CPU times: user 0 ns, sys: 878 µs, total: 878 µs
Wall time: 938 µs
16 1.2735252154501302
CPU times: user 392 µs, sys: 77 µs, total: 469 µs
Wall time: 397 µs
32 1.2741181443691536
CPU times: user 934 µs, sys: 0 ns, total: 934 µs
Wall time: 862 µs
64 1.274209131297663
CPU times: user 2.71 ms, sys: 0 ns, total: 2.71 ms
Wall time: 2.76 ms
128 1.2742221200377764
CPU times: user 14.5 ms, sys: 3.86 ms, total: 18.4 ms
Wall time: 14.5 ms
256 1.2742238859485546
CPU times: user 84.7 ms, sys: 66 ms, total: 151 ms
Wall time: 77.5 ms
512 1.2742241184580805
CPU times: user 303 ms, sys: 124 ms, total: 427 ms
Wall time: 261 ms
1024 1.274224148449696
CPU times: user 1.37 s, sys: 225 ms, total: 1.59 s
Wall time: 1.05 s
2048 1.27

From the above, it seems that the norm of A is around `1.274224152`

Let us try to double check these digits with another method.

In [2]:
n = 1024
M = [[A(i,j) for i in range(n)] for j in range(n)]
import scipy
from scipy.linalg import norm as scipy_norm
print("timing scipy.linalg.norm")
%time print(scipy_norm(M,2))
print("timing numpy.linalg.norm")
from numpy.linalg import norm as np_norm
%time print(np_norm(M,2))
print("timing calculation of spectral norm 'by hand'")
from numpy import matrix, matmul
from scipy.linalg import eigh
from math import sqrt
M = matrix(M)
%time print(sqrt(max(eigh(matmul(M.T,M),eigvals_only=True))))

timing scipy.linalg.norm
1.274224148449696
CPU times: user 798 ms, sys: 86.9 ms, total: 885 ms
Wall time: 486 ms
timing numpy.linalg.norm
1.274224148449696
CPU times: user 864 ms, sys: 139 ms, total: 1 s
Wall time: 514 ms
timing calculation of spectral norm 'by hand'
1.2742241484496957
CPU times: user 332 ms, sys: 73.9 ms, total: 405 ms
Wall time: 213 ms


In the above cell, we use the definition of the spectral norm of a matrix

$ \Vert{A}\Vert_2 = \sqrt{\lambda_{\max}(A^T A)} $
where $\lambda_{\max}$ is the largest eigenvalue of the given matrix. This seems to be the fastest method and agrees with the methods provided by numpy and scipy - thus, we will use it on a larger submatrix of A and see if the result agrees with our brute force attempt above

In [3]:
from numpy import matrix, matmul, max, sqrt
from scipy.linalg import eigh
def spectral_norm(M):
    return sqrt(max(eigh(matmul(M.T,M),eigvals_only=True)))
for n in [2**i for i in range(13)]:
    %time print(n, norm([[A(i,j) for i in range(n)] for j in range(n)],2))

1 1.0
CPU times: user 733 µs, sys: 0 ns, total: 733 µs
Wall time: 581 µs
2 1.1833501765516565
CPU times: user 533 µs, sys: 21 µs, total: 554 µs
Wall time: 1.25 ms
4 1.2525373975167995
CPU times: user 372 µs, sys: 15 µs, total: 387 µs
Wall time: 395 µs
8 1.2700463058540765
CPU times: user 517 µs, sys: 0 ns, total: 517 µs
Wall time: 444 µs
16 1.2735252154501302
CPU times: user 542 µs, sys: 0 ns, total: 542 µs
Wall time: 463 µs
32 1.2741181443691536
CPU times: user 897 µs, sys: 11 µs, total: 908 µs
Wall time: 820 µs
64 1.274209131297663
CPU times: user 2.71 ms, sys: 0 ns, total: 2.71 ms
Wall time: 2.64 ms
128 1.2742221200377764
CPU times: user 11 ms, sys: 3.98 ms, total: 15 ms
Wall time: 12 ms
256 1.2742238859485546
CPU times: user 88.2 ms, sys: 58.9 ms, total: 147 ms
Wall time: 75 ms
512 1.2742241184580805
CPU times: user 306 ms, sys: 122 ms, total: 428 ms
Wall time: 261 ms
1024 1.274224148449696
CPU times: user 1.4 s, sys: 181 ms, total: 1.58 s
Wall time: 1.05 s
2048 1.2742241522691704
